In [1]:
import os
import random
from argparse import Namespace
from collections import Counter
import json
import string
import re
import random
import nltk
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm.auto

In [2]:
!pip install -q opendatasets

import opendatasets as od
import pandas as pd

if not os.path.isfile('genius-song-lyrics-with-language-information/song_lyrics.csv'):
   od.download('https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/') # insert ypu kaggle  username and key

In [3]:
lyric_database = pd.read_csv('genius-song-lyrics-with-language-information/song_lyrics.csv', engine='python')
lyric_database.head(5)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [4]:
def fix_lyrics(lyrics):
    parts = re.split(r"([\n\[\]\(\)])", lyrics)
    output = " ".join(filter(None, parts))
    output = re.sub(r"([?.,!:;])",'',output)
    output = re.sub(r"in'(?= \w|\.|,|$)","ing",output.lower())
    return output

db = lyric_database.sample(frac=0.005, random_state=random.randint(1,1000)) #this database has 3 million entries
db = (
    db
    .loc[db["language"] == "en"]
    .loc[db["tag"] != "misc"]
    .loc[db["tag"] != ""]
    .loc[db["tag"] != None]
    .loc[db["lyrics"] != None]
    .loc[db["lyrics"] != ""]
    .loc[db["lyrics"] != "[Instrumental]"]
)
columns_of_interest = ['id', 'title','artist', 'lyrics', 'tag']
db = db[columns_of_interest]
db["lyrics"] = db["lyrics"].apply(fix_lyrics)
db = db.sort_values(by="id")

genres = {0:'pop', 1:'rap', 2:'rock', 3:'rb', 4:'country'}
db['tag'] = db['tag'].replace(['pop','rap','rock','rb','country'],[0,1,2,3,4])
db.head(5)

print(len(db.index))
db.head(10)

16117


,id,title,artist,lyrics,tag
150,152,Put On Remix,Jeezy,[ intro young jeezy ] \n rest in peace homie \...,1
371,366,Its All About the Benjamins Remix,Puff Daddy & the Family,[ produced by d-dot ( the mad rapper ) ] \n \...,1
384,379,Ghetto Quran Forgive Me,50 Cent,[ intro ] \n uh huh uh huh uh huh \n southside...,1
399,393,Pussy Money Weed,Lil Wayne,[ intro ] \n i love her like— \n oh— oh yes i ...,1
449,426,Who Could Fuck Wit Me,J.R. Writer,[ hook ] \n who could fuck with me \n who coul...,1
584,561,Space Bound,Eminem,[ verse 1 eminem ] \n we touch i feel a rush w...,1
666,635,The Food,Common,[ intro common ] \n yeah it's been a long time...,1
1545,1528,Bucharest,Jupiter (Band),verse 1 \n i’m from that place you never heard...,1
1686,1667,Black Republican,Nas,[ produced by les & wyldfyer ] \n \n [ intro j...,1
1819,1826,Wipe Me Down,Boosie Badazz,[ intro foxx ] \n mr wipe down in this biatch ...,1


https://github.com/ThilinaRajapakse/simpletransformers
Multi-label text classification

In [5]:
epochs = [3, 6, 11]
learning_rates = [0.00001, 0.0001, 0.0001]
matrix= [[0,0,0],[0,0,0],[0,0,0]]
for y, ep in enumerate(epochs):
    for x, rate in enumerate(learning_rates):
        matrix[x][y] = [ep,rate]

print(matrix)

[[[3, 1e-05], [6, 1e-05], [11, 1e-05]], [[3, 5e-05], [6, 5e-05], [11, 5e-05]], [[3, 0.0001], [6, 0.0001], [11, 0.0001]]]


I'll be testing two hyperparameters, epoch count and learning rates. For the sake of time, I have limited the epochs to a maximum of 11, though a higher value could be very beneficial as well. 

I've set up a nested for loop to iterate through each of my parameters and create a model. Afterward, we will evaluate each of the models using the evaluation portion of the data and determine which values of the hyperparameters are best for our larger model. 

In [7]:
#!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

epochs = [3, 6, 11]
learning_rates = [0.00001, 0.0001, 0.001]
results = [[None,None,None],[None,None,None],[None,None,None]]

tiny_db = db.sample(n=500, random_state=random.randint(1,1000))
print(len(tiny_db.index))

train, test = train_test_split(tiny_db[["lyrics", "tag"]], test_size=0.2, random_state=42, shuffle=True)
train_df =  pd.DataFrame(train).reset_index(drop=True)
test_df =  pd.DataFrame(test).reset_index(drop=True)

for y, ep in enumerate(epochs):
    for x, rate in enumerate(learning_rates):
        
        print(f"model{y+x+1}\n=======================\n\ntraining model with {epochs[y]} epochs and {learning_rates[x]} learning rate\n\n=======================\n")
        
        model_args = ClassificationArgs(num_train_epochs=epochs[y], learning_rate=learning_rates[x], overwrite_output_dir=True) 
        model = ClassificationModel(
            'bert',
            'bert-base-cased',
            use_cuda = False,
            num_labels=5,
            args=model_args,
        )

        model.train_model(train_df, verbose=False)
        result, model_outputs, wrong_predictions = model.eval_model(test_df)
        print(f"{matrix[x][y]} results: {result}\n\n")
        results[x][y] = result

500
model1

training model with 3 epochs and 1e-05 learning rate



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\dandy\Documents\COMPUTER SCIENCE\CS-39AA\project_part_3\venv\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/400 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_5_2


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_5_2


Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'eval_loss': 1.2845421662697425}


[11, 0.0001] results: {'mcc': 0.0, 'eval_loss': 1.2845421662697425}



In [ ]:
for y, row in enumerate(results):
    for x, result in enumerate(row):
        print(f"{matrix[x][y]} results: {result}")

Now it's time for the real thing. With our hyperparams fine-tuned, we can run the AI on a larger scale. 

In [8]:
train, test = train_test_split(db[["lyrics", "tag"]], test_size=0.2, random_state=42, shuffle=True)
train_df =  pd.DataFrame(train).reset_index(drop=True)
test_df =  pd.DataFrame(test).reset_index(drop=True)

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=6, learning_rate=0.00005, overwrite_output_dir=True) #use_multiprocessing=False,use_multiprocessing_for_evaluation=False)

# Create a MultiLabelClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    use_cuda = False,
    num_labels=5,
    args=model_args,
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    test_df
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\dandy\Documents\COMPUTER SCIENCE\CS-39AA\project_part_3\venv\lib\site-packages\simpletransformers\classification\classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/12893 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_5_2


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/1612 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
C:\Users\dandy\Documents\COMPUTER SCIENCE\CS-39AA\project_part_3\venv\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3224 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_5_2


Running Evaluation:   0%|          | 0/403 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.4606410655140295, 'eval_loss': 2.4615527489998206}


In [9]:
print(result)

{'mcc': 0.4606410655140295, 'eval_loss': 2.4615527489998206}


Now that the AI is completely trained, let's give it some songs and see what it predicts their genre as. Let's start with the song we tested in part 1, "Do I Wanna Know?" by Arctic Monkeys.

In [23]:
# = lyric_database.sample(n=50, random_state=random.randint(1,1000))
evaluation_db = (lyric_database.loc[lyric_database["id"] == 154314])
evaluation_db = evaluation_db[["lyrics", "tag"]].reset_index(drop=True)
evaluation_db["lyrics"] = evaluation_db["lyrics"].apply(fix_lyrics)

evaluation_db['tag'] = evaluation_db['tag'].replace(['pop','rap','rock','rb','country'],[0,1,2,3,4])
print(evaluation_db.head())

predictions, raw_outputs = model.predict(evaluation_db)
print(predictions)

                                              lyrics  tag
0  [ verse 1 ] \n have you got colour in your che...    2


KeyError: 0